In [ ]:
import covidcast
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

%cd ../code
from model import Model

%cd ../notebooks

In [ ]:
#Load data

start_date = datetime(2020, 10, 15)
end_date = datetime(2020, 12, 15)
    
X_df = covidcast.signal(
        'fb-survey', 
        'smoothed_cli',
        start_date, end_date,
        geo_type='state')

Y_df = covidcast.signal(
        'indicator-combination',
        'confirmed_7dav_incidence_num',
        start_date, end_date,
        geo_type='state')

In [ ]:
#Process data for use in Tensorflow

states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga',
       'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me',
       'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm',
       'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn',
       'tx', 'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']

df = X_df.sort_values(['geo_value', 'time_value'])
df = df[df['geo_value'].isin(states)]
X = pd.pivot_table(df,index='geo_value',columns=df.groupby(['geo_value']).cumcount().add(1), values='value').values
X = tf.expand_dims(X,axis=0) #shape=(1,number of geo_values, time series values)
X = tf.expand_dims(X,axis=-1) #shape=(1,number of geo_values, time series values, 1)
X = tf.cast(X, dtype=tf.float32)

df = Y_df.sort_values(['geo_value', 'time_value'])
df = df[df['geo_value'].isin(states)]
Y = pd.pivot_table(df,index='geo_value',columns=df.groupby(['geo_value']).cumcount().add(1), values='value').values
Y = tf.expand_dims(Y,axis=0) #shape=(1,number of geo_values, time series values)
Y = tf.expand_dims(Y,axis=-1) #shape=(1,number of geo_values, time series values, 1)
Y = tf.cast(Y, dtype=tf.float32)

In [ ]:
#Initialize model

#You can apply constraints directly to layers by setting
#the kernel_constraints parameter for a specific layer
#
#https://www.tensorflow.org/api_docs/python/tf/keras/constraints

p=30
m=X.shape[1] #number of geo_values
kernel_constraint = tf.keras.constraints.NonNeg()
lr=1

model = Model(
    p=p, 
    m=m,
    kernel_constraint=kernel_constraint
)
optimizer = tf.keras.optimizers.Adam(lr=lr)

def MSE(y_true, y_pred):
    return tf.reduce_mean((y_true-y_pred)**2)
loss=MSE

#This just monitors the specified quantity
#and creates a checkpoint when the monitored quantity is 
#lower than the previous recorded value

MODEL_PATH = "model.ckpt" #Please change the model path
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_PATH,
    verbose=0,
    monitor="MSE",
    mode='min',
    save_best_only=True,
    save_weights_only=True
)
callbacks = [
    checkpoint_callback,
]

model.compile(
    optimizer=optimizer,
    loss=loss, 
    metrics=[loss],
)

In [ ]:
#Training
hist = model.fit(
    x=X,
    y=Y,
    callbacks=callbacks,
    epochs=1000,
)

In [ ]:
#Evaluation
model.load_weights(MODEL_PATH).expect_partial() #load best weights from training